In [84]:
import requests
from bs4 import BeautifulSoup
import re

In [85]:
url='https://hojemacau.com.mo/'
headers={'USER-AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36 Edg/128.0.0.0'}
r=requests.get(url, headers=headers)
r

<Response [200]>

In [86]:
soup=BeautifulSoup(r.content)
lst_post=soup.find_all('h3', {'class': re.compile('post_title')})
restructured_posts=[]
for post in lst_post:
    dict_post={}
    dict_post['title']=post.a.text
    dict_post['link']=post.a.get('href')
    restructured_posts.append(dict_post)


In [89]:
lst_post=soup.find_all('div', {'class': 'jeg_postblock_content'})
# post=lst_post[0]

def get_article(url):
    headers={'USER-AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36 Edg/128.0.0.0'}
    r=requests.get(url, headers=headers)
    soup=BeautifulSoup(r.content)
    if soup.find('time', {'class': 'entry-date published updated'}):
        date=soup.find('time', {'class': 'entry-date published updated'}).get('datetime')
    else:
        date=None
        
    if soup.find('div', {'class': 'entry-content'}):
        content=soup.find('div', {'class': 'entry-content'}).text
    else:
        content=None
    return date, content

restructured_posts=[]
for idx, post in enumerate(lst_post):
    if post.find('div', {'class': re.compile('post_category')}):
        cat=post.find('div', {'class': re.compile('post_category')}).a.text
    else:
        cat=None

    if post.find('div', {'class': re.compile('author')}):
        author=post.find('div', {'class': re.compile('author')}).a.text.strip()
    else:
        author=None
    
    if post.find('h2'):
        head='h2'
    elif post.find('h3'):
        head='h3' 
    title=post.find(head, {'class': re.compile('post_title')}).a.text
    link=post.find(head, {'class': re.compile('post_title')}).a.get('href')
    date, content=get_article(link)

    dict_post={
        'title': title,
        'category': cat,
        'author': author,
        'link': link,
        'date': date,
        'content': content
    }
    restructured_posts.append(dict_post)

In [90]:
restructured_posts[0]

{'title': 'Associação Chinesa de Futebol bane 43 pessoas por corrupção',
 'category': 'China / Ásia',
 'author': None,
 'link': 'https://hojemacau.com.mo/2024/09/10/associacao-chinesa-de-futebol-bane-43-pessoas-por-corrupcao/',
 'date': '2024-09-10T22:36:38+08:00',
 'content': '\nA Associação Chinesa de Futebol (CFA) baniu 43 pessoas de participarem em actividades relacionadas com o futebol, devido ao seu envolvimento em manipulação de resultados e outras formas de corrupção.\nZhang Xiaopeng, um alto funcionário da polícia chinesa, divulgou ontem em conferência de imprensa os pormenores de uma “investigação de dois anos que revelou uma série de casos de apostas online, viciação de resultados e subornos”, segundo a agência noticiosa oficial Xinhua.\nSegundo a Xinhua, a investigação envolveu 120 jogos dos campeonatos chineses, 128 suspeitos de crimes e 41 clubes. Entre as pessoas banidas, 38 eram jogadores e cinco eram funcionários de vários clubes. Os antigos jogadores da selecção chine

In [92]:

import xml.etree.ElementTree as ET

# Create the root of the RSS feed
rss = ET.Element('rss', version='2.0')
channel = ET.SubElement(rss, 'channel')

# Add channel elements
ET.SubElement(channel, 'title').text = 'Hoje Macau'
ET.SubElement(channel, 'link').text = url
ET.SubElement(channel, 'description').text = 'Hoje Macau RSS'

# Add items
for item in restructured_posts:
    item_elem = ET.SubElement(channel, 'item')
    ET.SubElement(item_elem, 'title').text = item['title']
    ET.SubElement(item_elem, 'link').text = item['link']
    ET.SubElement(item_elem, 'description').text = item['content']
    ET.SubElement(item_elem, 'pubDate').text = item['date']

# Convert to string and save to an XML file
rss_feed = ET.tostring(rss, encoding='utf-8', method='xml').decode()
with open('Hoje_Macau.xml', 'w', encoding='utf-8') as xml_file:
    xml_file.write(rss_feed)